# Parse and Analyze GPS Data

Brett Deaton - Summer 2021

This notebook translates and analyzes the raw data stream from a GPS receiver. The raw data stream is read in from the file `nmea-data_stream.csv`, represented in the [NMEA-0183 format](https://navspark.mybigcommerce.com/content/NMEA_Format_v0.1.pdf). We'll use only the Global Positioning System Fix Data line denoted `GNGGA`. For projection, we'll treat the earth as a sphere with the [WGS84](https://en.wikipedia.org/wiki/World_Geodetic_System) radius, 637813 m.

The GPS receiver is the [u-blox SAM-M8Q](https://www.u-blox.com/en/product/sam-m8q-module). You can buy one already attached to a PCB by [sparkfun](https://www.digikey.com/en/products/detail/sparkfun-electronics/GPS-15210/10064422), already optimized with a copper-plane serving as an antenna.

### Set up

In [ ]:
import csv # to quickly read in comma-separated values
import matplotlib.pyplot as plt # to visually inspect the data
import statistics as st # for data summary
from math import cos, pi # for projection

In [ ]:
# read fix data from csv and convert to 2D list of strings
filename = "nmea-data_stream.csv"
points = []
num_bad_points = 0
with open(filename) as f:
    reader = csv.reader(f)
    for row in reader:
        if "GGA" in row[0]: # we're only interested in $GNGGA rows
            if row[6]!="0": # only keep points with fixes
                points.append(row[1:]) # drop leading "$GNGGA"
            else:
                num_bad_points += 1

In [ ]:
# look at the first and last points, for a sanity check
for point in (points[0], points[-1]):
    for x in point:
        print(x if x else "BLANK", end=" ")
    print()

### Extract and Convert Data

The NMEA-0183 format:

| index | meaning             | string format
|-------|---------------------|---------------
| 0     | UTC time            | hhmmss.s
| 1     | latitude            | ddmm.m
| 2     | N/S indicator       | ["N","S"]
| 3     | longitude           | dddmm.m
| 4     | E/W indicator       | ["E","W"]
| 5     | GPS quality         | [0,1,2]
| 6     | number of sats      | n
| 7     | horizontal DOP      | x.x
| 8     | altitude (m)        | x.x
| 9     | "M"
| 10    | geoid-ellipsoid (m) | x.x
| 11    | "M"
| 12    | DGPS station ID     | n
| 13    | checksum            | *0x

In [ ]:
# timestamps, in hrs since midnight
times = []
for x in points:
    times.append(float(x[0][:2]) +     # hr
                 float(x[0][2:4])/60 + # min
                 float(x[0][4:])/3600) # sec

# latitudes, in degrees north
lats = []
for x in points:
    sign = 1 if x[2]=="N" else -1
    lats.append(sign*int(x[1][:2]) + # deg
                sign*float(x[1][2:])/60)  # min

# longitudes, in degrees east
longs = []
for x in points:
    sign = 1 if x[4]=="E" else -1
    longs.append(sign*int(x[3][:3]) + # deg
                 sign*float(x[3][3:])/60)  # min

# altitudes, in meters above mean sea level
alts = []
for x in points:
    alts.append(float(x[8])) # altitude

In [ ]:
# GPS quality {0,1,2}
quals = []
for x in points:
    quals.append(int(x[5]))

# numbers of satellites used for position fix
fix_num = []
for x in points:
    fix_num.append(int(x[6]))

# horizontal dilutions of precision [0-100]
hdops = []
for x in points:
    hdops.append(float(x[7]))

### Summarize the Track

In [ ]:
MEAN_LAT = st.mean(lats)
MEAN_LONG = st.mean(longs)
print("Summary of track, with +- indicating stdev:")
if num_bad_points > 0:
    print(f"  Note, we dropped {num_bad_points} points without fixes")
print(f"  lats         {MEAN_LAT:.4f} +- {st.pstdev(lats):.5f} deg")
print(f"  longs        {MEAN_LONG:.4f} +- {st.pstdev(longs):.5f} deg")
print(f"  alts         min={min(alts):.0f} m, max={max(alts):.0f} m")
print(f"  time range   [ {times[0]:.4f} - {times[-1]:.4f} ] hrs since midnight UTC")
print(f"  qualities    min={min(quals)}, max={max(quals)}")
print(f"  num sats     min={min(fix_num)}, max={max(fix_num)}")
print(f"  HDOP         min={min(hdops)}, max={max(hdops)}")

### Visualize the Track

In [ ]:
# altitude vs time
plt.plot(times, alts, ".r")
plt.xlabel("time (hrs)")
plt.ylabel("altitude (m)")
plt.title("Altitude vs Time")
plt.savefig(filename.strip(".csv")+"-altitude.pdf")
plt.show()

In [ ]:
# jitter in residual latitudes and longitudes
plt.plot(times, [d-MEAN_LAT for d in lats],
         times, [d-MEAN_LONG for d in longs])
plt.legend([f"latitude - {MEAN_LAT:.3f}",
            f"longitude - {MEAN_LONG:.3f}"])
plt.xlabel("time (hrs)")
plt.ylabel("residual (deg)")
plt.title("Lat & Long vs Time")
plt.savefig(filename.strip(".csv")+"-lat_long.pdf")
plt.show()

In [ ]:
# show track in azimuthal projection centered on the mean position
RADIUS = 637813 # WGS84 radius (m)
pos_ns = [RADIUS*(th-MEAN_LAT)*pi/180 for th in lats]
pos_ew = [RADIUS*cos(MEAN_LAT)*(ph-MEAN_LONG)*pi/180 for ph in longs]
plt.plot(pos_ew, pos_ns)
plt.xlabel("E/W position (m)")
plt.ylabel("N/S position (m)")
plt.title("Projected Track Centered on Mean Position")
plt.savefig(filename.strip(".csv")+"-position.pdf")
plt.show()

### To Do

Tasks left to complete:
* project to (x,y) position in meters
* look for patterns in GPS quality indicator, i.e why some fixes were unavailable
* find correlation between number of satellites used for fixes and jitter in position
* compute checksums of each row and compare to recorded checksum